In [ ]:
#default_exp data.load

In [ ]:
#export
from mantisshrimp.all import *

# Data load
> Core modifications for making the dataloader work

## DataLoader

In [ ]:
#export
def _zip_collate(t): return Tuple2(zip(*t))
def _zip_convert(t): raise NotImplementedError

In [ ]:
#export
def detect_batch_to_samples(b, max_n=10): return L(b).zip()[:max_n]

In [ ]:
#export
class DetectDataLoader(TfmdDL):
    def create_batch(self, b): return (_zip_collate, _zip_convert)[self.prebatched](b)
    
    def _decode_batch(self, b, max_n=9, full=True):
        f = self.after_item.decode
        f = compose(f, partial(getattr(self.dataset,'decode',noop), full = full))
        return L(detect_batch_to_samples(b, max_n=max_n)).map(f)
    
    def show_batch(self, b=None, max_n=9, ctxs=None, show=True, unique=False, **kwargs):
        if unique:
            old_get_idxs = self.get_idxs
            self.get_idxs = lambda: Inf.zeros
        if b is None: b = self.one_batch()
        if not show: return self._pre_show_batch(b, max_n=max_n)
        show = show_batch[type(b[0][0]), type(b[1][0])]
        pb = self._pre_show_batch(b, max_n=max_n)
        show(*pb, ctxs=ctxs, max_n=max_n, **kwargs)
        if unique: self.get_idxs = old_get_idxs

In [ ]:
dset = np.arange(20).reshape(10,2)

In [ ]:
class Letter:
    def __init__(self, v): self.v = v
    def __repr__(self): return f'<Letter: {self.v.__repr__()}>'
    def __eq__(self, other): return self.v == other

In [ ]:
letters = L(string.ascii_lowercase, use_list=True).map(Letter)
numbers = L(range_of(letters))
dset = list(zip(letters,numbers)) 

In [ ]:
dls = DataLoaders.from_dsets(dset, bs=3, dl_type=DetectDataLoader, shuffle=False)
b = first(dls.train)

In [ ]:
test_eq(b, (Tuple(letters[:3]), Tuple(numbers[:3])))

In [ ]:
test_eq(type(b[0][0]), Letter)

In [ ]:
test_eq(detect_batch_to_samples(b, 2), [(letters[0], 0), (letters[1], 1)])

In [ ]:
test_eq(find_bs(b[0]), 3)

In [ ]:
# cannot be tested where cuda is not available
# dset = [[tensor(1), tensor(2)], [tensor(1), tensor(2)]]
# dls = DataLoaders.from_dsets(dset, bs=2, dl_type=DetectDataLoader, device='cuda')
# xb,yb = dls.one_batch()
# xb[0].is_cuda

In [ ]:
#hide
for b in progress_bar(dls.train): pass

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_fastai_core.ipynb.
Converted 03_data.load.ipynb.
Converted 04_data.core.ipynb.
Converted 06_callback.ipynb.
Converted 08_learner.ipynb.
Converted index.ipynb.
